<a href="https://colab.research.google.com/github/gogzicole/Hamoye-Data-science/blob/master/Data_munging_using_pandas_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import json

**loading data as pandas dataframe**

In [26]:
df = pd.read_html('https://en.wikipedia.org/w/index.php?title=Fortune_Global_500&oldid=855890446', header=0)[0]
df

,Rank,Company,Country,Industry,Revenue in USD
0,1,Walmart,United States,Retail,$500 billion
1,2,State Grid,China,Power,$349 billion
2,3,Sinopec Group,China,Petroleum,$327 billion
3,4,China National Petroleum,China,Petroleum,$326 billion
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion
5,6,Toyota Motor,Japan,Automobiles,$265 billion
6,7,Volkswagen,Germany,Automobiles,$260 billion
7,8,BP,United Kingdom,Petroleum,$245 billion
8,9,Exxon Mobil,United States,Petroleum,$244 billion
9,10,Berkshire Hathaway,United States,Products,$242 billion


**manually creating data with similar field as previous data**

In [29]:
other_data = [
    {"name": "Walmart",
     "employees": 2300000,
     "year founded": 1962
    },
    {"name": "State Grid Corporation of China",
     "employees": 927839,
     "year founded": 2002},
    {"name": "China Petrochemical Corporation",
     "employees":358571,
     "year founded": 1998
     },
    {"name": "China National Petroleum Corporation",
     "employees": 1636532,
     "year founded": 1988},
    {"name": "Toyota Motor Corporation",
     "employees": 364445,
     "year founded": 1937},
    {"name": "Volkswagen AG",
     "employees": 642292,
     "year founded": 1937},
    {"name": "Royal Dutch Shell",
     "employees": 92000,
     "year founded": 1907},
    {"name": "Berkshire Hathaway Inc.",
     "employees":377000,
     "year founded": 1839},
    {"name": "Apple Inc.",
     "employees": 123000,
     "year founded": 1976},
    {"name": "Exxon Mobile Corporation",
     "employees": 69600,
     "year founded": 1999},
    {"name": "BP plc",
     "employees": 74000,
     "year founded": 1908}
]

**defining a function that strips alphabets from numerals and coverts to float**

In [32]:
def convert_revenue(x):
    return float(x.lstrip('$').rstrip('billion')) * 1e9

assert convert_revenue('$500 billion') == 500e9

**using the merge function in Pandas**

**1. Renaming df[company]** 

In [10]:
new_names = dict(zip(df['Company'].unique(),['Walmart', 'State Grid Corporation of China',
                 'China Petrochemical Corporation','China National Petroleum Corporation',
                 'Royal Dutch Shell','Toyota Motor Corporation', 
                 'Volkswagen AG','BP plc','Exxon Mobile Corporation',
                 'Berkshire Hathaway Inc.']))

In [11]:
df['Company'] = df['Company'].replace(new_names)
df

,Rank,Company,Country,Industry,Revenue in USD
0,1,Walmart,United States,Retail,$500 billion
1,2,State Grid Corporation of China,China,Power,$349 billion
2,3,China Petrochemical Corporation,China,Petroleum,$327 billion
3,4,China National Petroleum Corporation,China,Petroleum,$326 billion
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion
5,6,Toyota Motor Corporation,Japan,Automobiles,$265 billion
6,7,Volkswagen AG,Germany,Automobiles,$260 billion
7,8,BP plc,United Kingdom,Petroleum,$245 billion
8,9,Exxon Mobile Corporation,United States,Petroleum,$244 billion
9,10,Berkshire Hathaway Inc.,United States,Products,$242 billion


In [12]:
df2 = DataFrame(other_data)
df2 = df2.rename(columns = {'name':'Company'})
df2

,Company,employees,year founded
0,Walmart,2300000,1962
1,State Grid Corporation of China,927839,2002
2,China Petrochemical Corporation,358571,1998
3,China National Petroleum Corporation,1636532,1988
4,Toyota Motor Corporation,364445,1937
5,Volkswagen AG,642292,1937
6,Royal Dutch Shell,92000,1907
7,Berkshire Hathaway Inc.,377000,1839
8,Apple Inc.,123000,1976
9,Exxon Mobile Corporation,69600,1999


**Performing inner merge to drop rows that dont match**

In [13]:
df4 = pd.merge(df,df2, on = 'Company',how = 'inner')
df4

,Rank,Company,Country,Industry,Revenue in USD,employees,year founded
0,1,Walmart,United States,Retail,$500 billion,2300000,1962
1,2,State Grid Corporation of China,China,Power,$349 billion,927839,2002
2,3,China Petrochemical Corporation,China,Petroleum,$327 billion,358571,1998
3,4,China National Petroleum Corporation,China,Petroleum,$326 billion,1636532,1988
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion,92000,1907
5,6,Toyota Motor Corporation,Japan,Automobiles,$265 billion,364445,1937
6,7,Volkswagen AG,Germany,Automobiles,$260 billion,642292,1937
7,8,BP plc,United Kingdom,Petroleum,$245 billion,74000,1908
8,9,Exxon Mobile Corporation,United States,Petroleum,$244 billion,69600,1999
9,10,Berkshire Hathaway Inc.,United States,Products,$242 billion,377000,1839


**converting revenues column from strings and integers to just integers**

In [16]:
df4['revenue'] = df['Revenue in USD'].apply(convert_revenue)
df4

,Rank,Company,Country,Industry,Revenue in USD,employees,year founded,revenue
0,1,Walmart,United States,Retail,$500 billion,2300000,1962,5.000000e+11
1,2,State Grid Corporation of China,China,Power,$349 billion,927839,2002,3.490000e+11
2,3,China Petrochemical Corporation,China,Petroleum,$327 billion,358571,1998,3.270000e+11
3,4,China National Petroleum Corporation,China,Petroleum,$326 billion,1636532,1988,3.260000e+11
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion,92000,1907,3.120000e+11
5,6,Toyota Motor Corporation,Japan,Automobiles,$265 billion,364445,1937,2.650000e+11
6,7,Volkswagen AG,Germany,Automobiles,$260 billion,642292,1937,2.600000e+11
7,8,BP plc,United Kingdom,Petroleum,$245 billion,74000,1908,2.450000e+11
8,9,Exxon Mobile Corporation,United States,Petroleum,$244 billion,69600,1999,2.440000e+11
9,10,Berkshire Hathaway Inc.,United States,Products,$242 billion,377000,1839,2.420000e+11


In [25]:
df4['rev per employee'] = (df4['revenue']/df4['employees']).astype(float)
df4.sort_values(by = 'rev per employee', ascending = False)


,Rank,Company,Country,Industry,Revenue in USD,employees,year founded,revenue,rev per employee
8,9,Exxon Mobile Corporation,United States,Petroleum,$244 billion,69600,1999,2.440000e+11,3.505747e+06
4,5,Royal Dutch Shell,Netherlands,Petroleum,$312 billion,92000,1907,3.120000e+11,3.391304e+06
7,8,BP plc,United Kingdom,Petroleum,$245 billion,74000,1908,2.450000e+11,3.310811e+06
2,3,China Petrochemical Corporation,China,Petroleum,$327 billion,358571,1998,3.270000e+11,9.119533e+05
5,6,Toyota Motor Corporation,Japan,Automobiles,$265 billion,364445,1937,2.650000e+11,7.271330e+05
9,10,Berkshire Hathaway Inc.,United States,Products,$242 billion,377000,1839,2.420000e+11,6.419098e+05
6,7,Volkswagen AG,Germany,Automobiles,$260 billion,642292,1937,2.600000e+11,4.048003e+05
1,2,State Grid Corporation of China,China,Power,$349 billion,927839,2002,3.490000e+11,3.761428e+05
0,1,Walmart,United States,Retail,$500 billion,2300000,1962,5.000000e+11,2.173913e+05
3,4,China National Petroleum Corporation,China,Petroleum,$326 billion,1636532,1988,3.260000e+11,1.992017e+05
